## Algoritmo desegunda optimización 



In [11]:
import json, os, logging, math
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional

# — Configuración —
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
INPUT = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_second_optimization.json"
OUTPUT = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/third_stage_optimized_relay_values.json"

# IEC y límites
K, N, CTI = 0.14, 0.02, 0.2
MIN_TDS, MAX_TDS = 0.05, 1.0
MIN_PICKUP = 0.05
MAX_PICKUP_FACTOR = 0.7
EPS = 1e-3
MAX_ITER = 200
AGG_MT = -2.0
AGG_TDS_BACK, AGG_TDS_MAIN = 1.15, 0.85
NORM_TDS_BACK, NORM_TDS_MAIN = 0.02, 0.01

def calc_time(Ishc: float, pickup: float, TDS: float) -> Optional[float]:
    if pickup <= 0 or TDS < MIN_TDS or TDS > MAX_TDS: return None
    if pickup >= Ishc: return None
    M = Ishc / pickup
    if M <= 1: return None
    denom = M**N - 1
    if denom <= 0: return None
    t = TDS * (K/denom)
    return t if math.isfinite(t) and t>0 else None

def corrige_backup(relay_vals, Ishc_back, t_main):
    # fuerza pickup < Ishc y >0
    pu = min(Ishc_back*0.9, Ishc_back - EPS)
    pu = max(MIN_PICKUP, pu)
    tds = MIN_TDS
    tb = calc_time(Ishc_back, pu, tds)
    if tb and tb > 0:
        return tds, pu, tb
    # si sigue mal, sube TDS en pasos hasta encontrar algo
    steps = int((MAX_TDS - MIN_TDS)/NORM_TDS_BACK)+1
    for i in range(1, steps):
        tds_c = MIN_TDS + i*NORM_TDS_BACK
        tb = calc_time(Ishc_back, pu, tds_c)
        if tb and tb > 0:
            return tds_c, pu, tb
    return MIN_TDS, pu, None

def carga_escenarios(data):
    esc = {}
    for e in data:
        s = e['scenario_id']
        esc.setdefault(s, {'pairs':[], 'relays':set()})
        m, b = e['main_relay'], e['backup_relay']
        esc[s]['pairs'].append({
            'm': m['relay'], 'b': b['relay'],
            'Im': float(m['Ishc']), 'Ib': float(b['Ishc']),
            'Lm': m.get('line'), 'Lb': b.get('line')
        })
        esc[s]['relays'].update([m['relay'], b['relay']])
    return esc

def optimiza(scenario):
    # init
    vals = {r:{'TDS':MIN_TDS,'pickup':MIN_PICKUP} for r in scenario['relays']}
    # garantizar pickup<Ishc
    for p in scenario['pairs']:
        for role, r in [('m','m'),('b','b')]:
            Ir = p['Im'] if role=='m' else p['Ib']
            pu = vals[p[role]]['pickup']
            if pu >= Ir:
                vals[p[role]]['pickup'] = min(Ir*0.9, Ir - EPS)
    # loop
    for _ in range(MAX_ITER):
        worst = 0
        tareas = []
        for p in scenario['pairs']:
            m, b = p['m'], p['b']
            tm = calc_time(p['Im'], vals[m]['pickup'], vals[m]['TDS'])
            tb = calc_time(p['Ib'], vals[b]['pickup'], vals[b]['TDS'])
            # si tb invalido o ≤0, corrige
            if tb is None or tb <= 0:
                tds_n, pu_n, tb_new = corrige_backup(vals[b], p['Ib'], tm or CTI)
                vals[b].update({'TDS':tds_n,'pickup':pu_n})
                tb = tb_new
            if tm is None or tb is None: continue
            mt = tb - tm - CTI
            tareas.append((m,b,mt))
            if mt < worst: worst = mt
        if worst >= 0: break
        # ajustes
        for m,b,mt in tareas:
            if mt >= 0: continue
            if mt < AGG_MT:
                vals[b]['TDS'] = min(MAX_TDS, vals[b]['TDS']*AGG_TDS_BACK)
                vals[m]['TDS'] = max(MIN_TDS, vals[m]['TDS']*AGG_TDS_MAIN)
            else:
                vals[b]['TDS'] = min(MAX_TDS, vals[b]['TDS']+NORM_TDS_BACK)
                vals[m]['TDS'] = max(MIN_TDS, vals[m]['TDS']-NORM_TDS_MAIN)
    # R1/R38 en L1-2
    for r in ('R1','R38'):
        otros = [p for p in scenario['pairs']
                 if (p['m']==r and p['Lb']!='L1-2') or (p['b']==r and p['Lm']!='L1-2')]
        if not otros: continue
        tds, pu = [], []
        for p in otros:
            otro = p['b'] if p['m']==r else p['m']
            tds.append(vals[otro]['TDS']); pu.append(vals[otro]['pickup'])
        vals[r]['TDS'] = sum(tds)/len(tds)
        vals[r]['pickup'] = sum(pu)/len(pu)
    return vals

def main():
    data = json.load(open(INPUT,'r'))
    esc = carga_escenarios(data)
    out = []
    for s,info in esc.items():
        logging.info(f"Optimizando {s}")
        res = optimiza(info)
        out.append({'scenario_id':s,
                    'timestamp':datetime.now(timezone.utc).isoformat().replace('+00:00','Z'),
                    'relay_values':res})
    os.makedirs(os.path.dirname(OUTPUT), exist_ok=True)
    json.dump(out, open(OUTPUT,'w',encoding='utf-8'), indent=2, ensure_ascii=False)
    logging.info(f"Guardado en {OUTPUT}")

if __name__=="__main__":
    main()

2025-04-21 16:23:13,117 - INFO - Optimizando scenario_1
2025-04-21 16:23:13,133 - INFO - Optimizando scenario_2
2025-04-21 16:23:13,152 - INFO - Optimizando scenario_3
2025-04-21 16:23:13,171 - INFO - Optimizando scenario_4
2025-04-21 16:23:13,188 - INFO - Optimizando scenario_5
2025-04-21 16:23:13,205 - INFO - Optimizando scenario_6
2025-04-21 16:23:13,223 - INFO - Optimizando scenario_7
2025-04-21 16:23:13,243 - INFO - Optimizando scenario_8
2025-04-21 16:23:13,262 - INFO - Optimizando scenario_9
2025-04-21 16:23:13,282 - INFO - Optimizando scenario_10
2025-04-21 16:23:13,301 - INFO - Optimizando scenario_11
2025-04-21 16:23:13,373 - INFO - Optimizando scenario_12
2025-04-21 16:23:13,409 - INFO - Optimizando scenario_13
2025-04-21 16:23:13,433 - INFO - Optimizando scenario_14
2025-04-21 16:23:13,451 - INFO - Optimizando scenario_15
2025-04-21 16:23:13,470 - INFO - Optimizando scenario_16
2025-04-21 16:23:13,487 - INFO - Optimizando scenario_17
2025-04-21 16:23:13,505 - INFO - Optimiz